# Linear Model Guide
In statistics, linear regression is a linear approach to modeling the relationship between a scalar response (or dependent variable) and one or more explanatory variables (or independent variables). The case of one explanatory variable is called simple linear regression. For more than one explanatory variable, the process is called multiple linear regression. This term is distinct from multivariate linear regression, where multiple correlated dependent variables are predicted, rather than a single scalar variable.

### Objective for competition: https://www.kaggle.com/c/house-prices-advanced-regression-techniques

* Predict the value of homes.
* Exploratory data analysis.
* Feature engineering.
* Create your own regression models.
* Compare metrics.
* Choose a model.
* To predict.

# Import

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas_summary as ps

from sklearn.base import BaseEstimator
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer as Imputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, SGDRegressor, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import gc

from scipy.stats import norm

In [ ]:
sns.set()
warnings.simplefilter('ignore')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

### We collect the metrics of all the algorithms

In [ ]:
all_metrics = []

# Read data

In [ ]:
folder = '/kaggle/input/house-prices-advanced-regression-techniques/'
train_df = pd.read_csv(folder+'train.csv')
test_df = pd.read_csv(folder+'test.csv')
sub_df = pd.read_csv(folder+'sample_submission.csv')

In [ ]:
print('train: ', train_df.shape)
print('test: ', test_df.shape)
print('sample_submission: ', sub_df.shape)

In [ ]:
train_df.head()

In [ ]:
train_df.info()

In [ ]:
test_df.head()

In [ ]:
test_df.info()

In [ ]:
sub_df.head()

# Describe the data

In [ ]:
dfs = ps.DataFrameSummary(train_df)
print('categoricals: ', dfs.categoricals.tolist())
print('numerics: ', dfs.numerics.tolist())
dfs.summary()

In [ ]:
dfs = ps.DataFrameSummary(test_df)
print('categoricals: ', dfs.categoricals.tolist())
print('numerics: ', dfs.numerics.tolist())
dfs.summary()

In [ ]:
train_df.drop('Id', inplace=True, axis=1)
test_df.drop('Id', inplace=True, axis=1)

# Get target

In [ ]:
ps.DataFrameSummary(train_df[['SalePrice']]).summary().T

In [ ]:
plt.figure(figsize=(12,5))
sns.distplot(train_df['SalePrice'] , fit=norm);

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(train_df['SalePrice'])

#Now plot the distribution
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('SalePrice distribution');

### Let's logarithm the value of the house

In [ ]:
train_df['SalePrice'] = np.log(train_df['SalePrice'])

In [ ]:
ps.DataFrameSummary(train_df[['SalePrice']]).summary().T

In [ ]:
plt.figure(figsize=(12,5))
sns.distplot(train_df['SalePrice'] , fit=norm);

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(train_df['SalePrice'])

#Now plot the distribution
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('SalePrice distribution');

In [ ]:
y = train_df['SalePrice']
train_df.drop(['SalePrice'], axis=1, inplace=True)

# Check some Null's

In [ ]:
def missing_values_table(df, info=True):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        if info:
            print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
                "There are " + str(mis_val_table_ren_columns.shape[0]) +
                  " columns that have missing values.")
        return mis_val_table_ren_columns

In [ ]:
missing_values_table(train_df).T

In [ ]:
missing_values_table(test_df).T

# Drop columns with a lot of NaNs (more than 75%)

In [ ]:
miss_df = missing_values_table(train_df)
drops = miss_df[miss_df['% of Total Values'] >75].index.tolist()
train_df.drop(drops, inplace=True, axis=1)
test_df.drop(drops, inplace=True, axis=1)

In [ ]:
print('Dropped: ', drops)

In [ ]:
del miss_df
gc.collect();

### Categoricals & Numerics columns

In [ ]:
dfs = ps.DataFrameSummary(train_df)
cat_cols = dfs.categoricals.tolist() + dfs.bools.tolist()
num_cols = dfs.numerics.tolist()

# Processing categoricals variables

In [ ]:
train_df[cat_cols] = train_df[cat_cols].fillna('?')
test_df[cat_cols] = test_df[cat_cols].fillna('?')

## Make mean target encoding for categorical feature

Let us consider the above table (A simple binary classification). 

$$ MeanTargetEnc_i = {((GlobalMean * C) + (Mean_i * Size)) \over (C + Size)} $$

Instead of finding the mean of the targets, we can also focus on median and other statistical correlations….These are broadly called target encodings

In [ ]:
class MeanEncoding(BaseEstimator):
    """   In Mean Encoding we take the number 
    of labels into account along with the target variable 
    to encode the labels into machine comprehensible values    """
    
    def __init__(self, feature, C=0.1):
        self.C = C
        self.feature = feature
        
    def fit(self, X_train, y_train):
        
        df = pd.DataFrame({'feature': X_train[self.feature], 'target': y_train}).dropna()
        
        self.global_mean = df.target.mean()
        mean = df.groupby('feature').target.mean()
        size = df.groupby('feature').target.size()
        
        self.encoding = (self.global_mean * self.C + mean * size) / (self.C + size)
    
    def transform(self, X_test):
        
        X_test[self.feature] = X_test[self.feature].map(self.encoding).fillna(self.global_mean).values
        
        return X_test
    
    def fit_transform(self, X_train, y_train):
        
        df = pd.DataFrame({'feature': X_train[self.feature], 'target': y_train}).dropna()
        
        self.global_mean = df.target.mean()
        mean = df.groupby('feature').target.mean()
        size = df.groupby('feature').target.size()
        self.encoding = (self.global_mean * self.C + mean * size) / (self.C + size)
        
        X_train[self.feature] = X_train[self.feature].map(self.encoding).fillna(self.global_mean).values
        
        return X_train

In [ ]:
for f in cat_cols:
    me = MeanEncoding(f, C=0.1)
    me.fit(train_df, y)
    train_df = me.transform(train_df)
    test_df = me.transform(test_df)

# Processing numerics variables

### Imputer with mean strategy

In [ ]:
imputer = Imputer(strategy="mean")
imputer.fit(train_df[num_cols])
train_df[num_cols] = imputer.transform(train_df[num_cols])
test_df[num_cols] = imputer.transform(test_df[num_cols])

# Check data again

In [ ]:
dfs = ps.DataFrameSummary(train_df)
dfs.summary()

In [ ]:
dfs = ps.DataFrameSummary(test_df)
dfs.summary()

In [ ]:
train_df.hist(figsize=(35, 30));

In [ ]:
train_corr = train_df.corr()
# plot the heatmap and annotation on it
fig, ax = plt.subplots(figsize=(18,18))
sns.heatmap(train_corr, xticklabels=train_corr.columns, yticklabels=train_corr.columns, annot=True, ax=ax);

### Ooops :)

In [ ]:
high_corr = pd.DataFrame(train_corr[(train_corr > 0.8) & (train_corr != 1)].fillna(0).sum(axis=0))
high_corr[high_corr[0]>0]

## Select columns with high correlation  to drop

In [ ]:
drops = ['Exterior2nd', '1stFlrSF', 'GrLivArea', 'Fireplaces', 'GarageCars', 'GarageCond', 'SaleType']
train_df.drop(drops, inplace=True, axis=1)
test_df.drop(drops, inplace=True, axis=1)

# Fit

Regression models are the simplest and at the same time effective machine learning models that give interpretable results.
However, using ready-made implementations from popular libraries, such as Scikit-Learn, with little thought about what is happening under the hood, this is one thing.
But to implement all the features of these algorithms with your own code is completely different.

In this project work, I plan to thoroughly understand the theoretical foundations of regression models (including regularized ones), how to optimize them (analytical solutions, various gradient descent implementations), implement these algorithms myself and compare them with the implementation in Scikit-Learn.

In the framework of this work, regression models will be considered that are used to predict continuous random variables (logistic regression is not considered).

# Linear regression models without regularization

### Linear Regression House Prices: Advanced Regression Techniques


___Linear regression calculates the weighted sum of input values (feature values) and free term .___

__The formula for predicting linear regression in the analytical form:__

$$\hat{y}_k = \theta_0 + \theta_1 x_{1}^{(k)} + \theta_2 x_{2}^{(k)} + ... + \theta_i x_{i}^{(k)} + ... + \theta_m x_{m}^{(n)}$$

Where:

$\hat{y}_k$ - calculated value obtained from the $k$ - observation vector using the model;

$\theta_i$ - value of the $i- parameter of the model;

$x_{i}^{(k)}$ - the value of the $i$- characteristic of the $k$- vector of the $X$ matrix of size $n$*$(m+1)$. 

___There are 2 types of training for this model:___
    
- the use of a direct equation in an analytical form that directly calculates the model parameters that are most suitable for a particular data set.
- application of the gradient descent method, which iteratively selects the model parameters, reducing the values of the loss function.

___Everything is clear with the model. How to train it? To teach this means to set her such parameters with which she will most accurately predict the values of the target variable on the training data .___

___What does "predict as accurately as possible" mean? What measure can be used?___

___There are many ways to measure the deviation of forecast values from real values. It is necessary to somehow evaluate the differences between the forecast and real values (residuals).
Here are some of the ways:___
    
__1. Summarize leftovers__

`Advantages`: easy to understand.

`Disadvantages`: terms with opposite signs can cancel each other and we will not receive information about the actual size of the deviation.

$$\sum_{k=1}^{n} (\hat{y}_k - y_k)$$

__2. Sum the remainder modules and divide by the number of forecasts__

`Advantages`: works if errors are distributed abnormally, easy to calculate.

`Disadvantages`: does not work, if the errors are distributed normally, the function of the sum of modules is not differentiable at zero.

$$\sum_{k=1}^n |\hat{y}_k - y_k|$$

_3. Sum the squares (or other positive even degrees) of the residuals and divide by the number of predictions__

ʻAdvantages`: it works regardless of the distribution of errors, the sum of squares function is differentiable.

`Disadvantages`: it is possible to use only positive even degrees, it is more difficult to calculate than the difference (the absolute value of the difference), in the presence of outliers the error value can“ explode ”, with an error value of 1 it is always 1 (1 to any degree is 1), large degrees are difficult to calculate.

$$\sum_{k=1}^n (\hat{y}_k - y_k)^2$$

__The choice of the loss function affects the result of our efforts to select the best parameters (coefficients $\theta$) .__

__As a rule, the value of the sum of losses (squared or modules, it doesn’t matter) is normalized to the number of observations (divided by the number of observations) to obtain the average value.__

__We will analyze the most common criteria.__

__Mean Absolute Error - MAE (Mean Absolute Error):__

$$ MAE(X,h_\theta) = \frac{1}{n}\sum_{k=1}^n |\hat{y}_k - y_k|$$

__Mean Squared Error - MSE (RMS Error):__

$$MSE(X,h_\theta) = \frac{1}{n}\sum_{k=1}^n (\theta^T X_k - y_k)^2 = \frac{1}{n}\sum_{k=1}^n (h_\theta(X_k) - y_k)^2$$

__Root Mean Squared Error - RMSE (square root of root mean square error):__

$$RMSE(X,h_\theta) = \sqrt{\frac{1}{n}\sum_{k=1}^n (h_\theta(X_k) - y_k)^2}$$

__$R^2$ (Share of predicted fluctuations in the total number of fluctuations in the data):__

$$R^2 (X,h_\theta) = 1 - \frac{\sum_{k=1}^n (h_\theta(X_k) - y_k)^2}{\sum_{k=1}^n (h_\theta(X_k) - \bar{y})^2} = 1 - \frac{RSS}{TSS}$$

In [ ]:
# scale data
scaler = StandardScaler().fit(train_df)
train_df = scaler.transform(train_df)
test_df = scaler.transform(test_df)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_df, y, test_size=0.2, random_state=10)

We got some values of the coefficients $\theta$, but we understand that they are obtained from data in which there is noise.

In [ ]:
# add a unit column to the left of the observation matrix (theta zero parameter will be multiplied by 1)
X_train_with_c = np.c_[np.ones((X_train.shape[0], 1)), X_train] 
theta_best_analytic = np.linalg.inv(X_train_with_c.T.dot(X_train_with_c)).dot(X_train_with_c.T).dot(y_train)
pd.DataFrame(theta_best_analytic).T

Now you can use our trained model for forecasts. Take the points from the test set and make a prediction.

In [ ]:
X_test_with_c = np.c_[np.ones((X_test.shape[0], 1)), X_test]
y_pred_simple = X_test_with_c.dot(theta_best_analytic)

Compare the result (y_pred) with real data (y_test).

We measure the RMSE indicator for our own implementation:

In [ ]:
all_metrics.append(['analytical_solution', mean_squared_error(y_test, y_pred_simple)])
mean_squared_error(y_test, y_pred_simple)

Compare our implementation of the algorithm with the LinearRegression method implemented in sklearn

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

Let's measure the RMSE for sklearn:

In [ ]:
all_metrics.append(['sklearn_LR', mean_squared_error(y_test, lr.predict(X_test))])
mean_squared_error(y_test, lr.predict(X_test))

Therefore, for real-world problems with hundreds and thousands of features, an approach with iterative optimization of parameters is used - Gradient Descent.

#### Testing the linear regression model for compliance with the criteria of the Gauss-Markov theorem:

A data model is correctly specified if the data has the following properties:

1. All $X_k$ are deterministic and not all are equal to each other (the matrix $X$ is deterministic, it contains real observations - the vectors $X_k$);

2. Model errors are not systematic, that is, the mathematical expectation of model errors is $ M [u] = 0 $, the variance of model errors is constant and equal to $Var[u]=\sigma^2$;

3. Errors are uncorrelated, that is, $M[u_i, u_j]=0 $ when $i$ is not equal to $j$ (pairwise correlation is zero).

__Then under these conditions the least-squares estimates are optimal in the class of linear unbiased estimates, in other words $\hat{\theta}=(X^TX)^{-1}X^Ty$ is the best estimate possible .__

In [ ]:
# error expectation
MU = np.round(np.sum((y_test - lr.predict(X_test)))/y_test.shape[0], 5)
MU

In [ ]:
# error variance
VAR = ((y_test - lr.predict(X_test)) - np.mean(y_test - lr.predict(X_test))).mean()
VAR

In [ ]:
# error variance is not generally constant
fig, ax = plt.subplots(figsize=(12,5))
plt.plot(((y_test - lr.predict(X_test)) - np.mean(y_test - lr.predict(X_test))))
fig.suptitle("Pic. Error spread relative to their mean value", \
            fontsize = 14, y = 1.03)
ax.axhline(y=MU, color='grey', ls='--');

We will write a function that will take pairs from the array with errors many times and at the end will calculate the expectation.

In [ ]:
def sample_corr(arr, it=10000):
    arr = arr.values.reshape(-1,1)
    val_list = []
    for iti in range(it):
        random_index = list(set(np.random.randint(arr.shape[0], size=2)))
        val_list.append(np.sum(arr[random_index, :]))
        
    return np.mean(val_list)

In [ ]:
sample_corr((y_test - lr.predict(X_test)))

# Gradient Descent House Prices: Advanced Regression Techniques

__The principle of the gradient descent method is to move along a function from some random point in the direction of the anti-gradient. At the same time, the step size (learning rate) is very important, which, if too large, will not allow us to step to the minimum, and if too small, we will go down to the minimum for a very long time .__

__If the signs have different scales, then the algorithm may converge slowly. We recommend scaling data before applying gradient descent .__

#### Batch gradient descent

To implement gradient descent in the case of a function with the number of parameters $\theta$ $i$>=2, you need to calculate the gradient (the vector of partial derivatives for each parameter $\theta_i$. The gradient shows how much the function changes (increases or decreases) with a small step over all $\theta$.

For the MSE loss function, the partial derivative with respect to $\theta_i$ will look like this:

$$MSE(X,h_\theta)_i' = \frac{d}{d\theta_i} MSE(X,h_\theta) = 
(\frac{1}{n}\sum_{k=1}^n (h_\theta(X_k) - y_k)^2)' =  (\frac{1}{n}\sum_{k=1}^n (\theta^T X_k - y_k)^2)' = 
\frac{2}{n}\sum_{k=1}^n (\theta^T X_k - y_k) \sum_{k=1}^m x_{i}^{(k)}$$ 

__Gradient (matrix of partial derivatives of the loss function for each parameter from $\theta_0$ to $\theta_i$) of the loss function $MSE$ will look like this:__

$$
grad_\theta MSE(X,h_\theta) = 
\left(\begin{array}{cc} 
\frac{d}{d\theta_0} MSE(X,h_\theta) \\
\frac{d}{d\theta_1} MSE(X,h_\theta) \\
... \\
\frac{d}{d\theta_i} MSE(X,h_\theta) \\
... \\
\frac{d}{d\theta_m} MSE(X,h_\theta)
\end{array}\right)
= \frac{2}{n} X^T (X  \theta^T - y)
$$ 

The algorithm is called __*batch gradient descent*__, because at each step it works with a complete package of training data. For real data sets, such an algorithm will work rather slowly.

Having calculated the gradient (the direction of the maximum growth of the function), we must take a “step” in the opposite direction (along the anti-gradient).
At the first step, from the randomly initialized parameters $\theta$ we subtract the obtained gradient and update the coefficient values in the vector $\theta$, at the next step we subtract the last calculated gradient from the most recent vector $\theta$, and so many, many times. In order to somehow normalize the step, we need to introduce
another coefficient is $\mu$, by which we will multiply the gradient. The $\mu$ parameter is called the __learning rate__.

__Calculation of the next step of the gradient descent:__

$$\theta^{next} = \theta^{current} - \mu grad_\theta MSE(X,h_{\theta^{current}})$$

Where:

$\theta^{current}$ - current value of the coefficient vector $\theta$;

$\theta^{next}$ - the following value of the coefficient vector $\theta$;

$MSE(X,h_{\theta^{current}})$ - value $MSE$ for function $h$ with the current set of coefficients $\theta$, which the observation matrix is passed as an argument $X$;

$\mu$ - learning speed.

`Advantages`: the algorithm allows you to find parameters not only in the case of linear regression (for which you can find these parameters analytically), but also for many other models, including neural networks. This advantage applies to all the options for implementing the gradient descent method listed in this paper.

`Disadvantages`: you need to carefully select the learning speed, for better convergence, you need to scale the data before applying the algorithm. Both of the latter drawbacks apply to all the options for implementing the gradient descent method listed in this paper.

__Now we will test this algorithm in practice. We have already created the training data. We write a function for gradient descent and then generate new points and make a forecast .__

In [ ]:
def gradient_descent(mu, x, y, params, numIterations, lf='MSE', prnt=False):
    """
     The function implements a batch gradient descent algorithm.
     mu - learning rate
     params - number of parameters, including free parameter
     numIterations - number of iterations (int)
     prnt - whether or not to print the calculation, if prnt = True, 
     then at every hundredth step the value of the loss function is displayed
     lf - loss function, default 'MSE', you can select 'MAE'
    """
    
    n = x.shape[0] #количество наблюдений в выборке
    theta = np.ones(params).reshape(params,1) # [ 1.  1. 1.] - начальные значения коэффициентов пусть будут равны 1
    x_transpose = x.transpose() # транспонированная матрица x
    for iter in range( 0, numIterations ):
        hypothesis = np.dot(x, theta) # матричное произведение
        loss = hypothesis - y.values.reshape(len(y),1) # значение остатка
        
        if lf=='MSE':
            J = np.sum(loss ** 2) / n  # функция потерь (квадраты)
            if prnt and (iter % 10000)==0:
                print( "iter %s | MSE: %.3f" % (iter, J) )
        
        elif lf=='MAE':
            J = np.sum(abs(loss)) / n  # функция потерь (модули)
            if prnt and (iter % 10000)==0:
                print( "iter %s | MAE: %.3f" % (iter, J) )
        
        gradient = np.dot(x_transpose, loss) * 2 / n         
        theta = theta - mu * gradient  # update
    
    return (theta)

In [ ]:
%%time
# add a unit column to the left of the observation matrix (theta zero parameter will be multiplied by 1)
X_train_with_c = np.c_[np.ones((X_train.shape[0], 1)), X_train] 
# run the gradient descent algorithm

theta_best_gd = gradient_descent(0.0001, X_train_with_c, y_train, params=X_train_with_c.shape[1], numIterations=100000, lf='MSE', prnt=True)

In [ ]:
# received parameters (gradient descent)
pd.DataFrame(np.round(theta_best_gd, 3)).T

In [ ]:
# received parameters (analytical solution)
pd.DataFrame(np.round(theta_best_analytic, 3)).T

## The obtained parameters are very far from those that we obtained analytically.

In [ ]:
# take new points from the test sample and apply the coefficients to calculate the function value from them,
# obtained by our gradient descent algorithm
X_test_with_c = np.c_[np.ones((X_test.shape[0], 1)), X_test] 
y_pred_gd = X_test_with_c.dot(theta_best_gd)

RMSE is also very close to sklearn and far from our analytical solution:

In [ ]:
all_metrics.append(['Gradient_Descent', mean_squared_error(y_test, y_pred_gd)])
mean_squared_error(y_test, y_pred_gd)

# Stochastic gradient descent House Prices: Advanced Regression Techniques

This modification of the algorithm allows you to bypass the problem that the batch method suffers, namely, the need to work with the entire set of training data.
At each step, the stochastic gradient descent selects one random sample from the training set and calculates the gradient only on the basis of this sample.
This greatly increases the speed of work, but instead of translating to the minimum of the loss function, we will observe a wandering indicator of the loss function, which will decrease only on average.
Having reached a minimum, the algorithm will continue to "rush" in its vicinity. The final values of the parameters $\theta$ will be good, but not optimal.

`Advantages`: the algorithm is computationally cheaper than the batch version, quickly gives good results, in the case of an irregular loss function (with many local extrema), the algorithm has good chances to jump out of a local minimum and get to a deeper local minimum or even to a global one.

`Disadvantages`: the final answer will not be optimal (the algorithm will not stop even if it gets to a minimum), it will most likely just be" good ".

One solution to the latter drawback is to gradually decrease the ___learning rate___. This approach is called ___simulated annealing___. A special function responsible for changing the learning speed is called the ___learning schedule___. As with the selection of the learning rate, there is no exact answer on how to set it, so with the rate of decrease in the learning rate you need to be careful not to stop ahead of time or to jump over the global minimum.

In [ ]:
def learning_schedule(val, p1=100, p2=50):
    """The function takes parameters as arguments
    p1 - parameter for the numerator (default is 100),
    p2 - parameter for the denominator (default is 50),
    val - value set by user
    and returns a simple conversion
    return p1/(p2 + val)
    """
    return p1/(p2 + val)

In [ ]:
def st_gradient_descent(x, y, params, num_epochs, num_iter, lf='MSE', prnt=False):
    """
     The function implements the stochastic gradient descent algorithm.
     mu - learning rate
     params - number of parameters, including free parameter
     num_epochs - number of eras
     num_iter - the number of iterations (int) in the era
     prnt - print or not calculations every 100 iterations
     lf - loss function, default 'MSE', you can select 'MAE'
     Without the learning_schedule () function, it won’t work.
    """
    
    n = x.shape[0] #количество наблюдений в выборке
    theta = np.ones(params).reshape(params,1) # [ 1.  1.  1.] - начальные значения коэффициентов пусть будут равны 1
    y = y.values.reshape(-1,1)
    lr=0.0001
    for epoch in range(num_epochs):
        for iteration in range(num_iter):
            random_index = np.random.randint(n)
            x_rand = x[random_index, :].reshape(1, params)
            y_rand = y[random_index, :]

            hypothesis = np.dot(x_rand, theta) # матричное произведение
            loss = hypothesis - y_rand.reshape(-1,1) # значение остатка

            if lf=='MSE':
                J = np.sum(loss ** 2) / n  # функция потерь (квадраты)
                if prnt and (iteration % 10000)==0:
                    print( "epoch %s | iter %s | MSE: %.3f" % (epoch, iteration, J) )

            elif lf=='MAE':
                J = np.sum(abs(loss)) / n  # функция потерь (модули)
                if prnt and (iteration % 10000)==0:
                    print( "iter %s | MAE: %.3f" % (iteration, J) )

            gradient = np.dot(x_rand.transpose(), loss) * 2 / n  
            lr = learning_schedule(val=100, p1=1, p2=10000)
            
            theta = theta - lr * gradient  # update
    
    return (theta)

Let's run our algorithm for 100 epochs with 2000 iterations (only 100,000 times) and measure the execution time.

In [ ]:
%%time
theta_best_sgd = st_gradient_descent(X_train_with_c, y_train, params=X_train_with_c.shape[1], num_epochs=100, num_iter=2000, lf='MSE', prnt=True)

In [ ]:
# received parameters (SGD)
pd.DataFrame(np.round(theta_best_sgd, 3)).T

`Comment:` as time measurement showed, stochastic gradient descent is faster than burst. Both implementations in the above examples updated $\theta$ parameters 100,000 times.

In [ ]:
X_test_with_c = np.c_[np.ones((X_test.shape[0], 1)), X_test] 
y_pred_sgd = X_test_with_c.dot(theta_best_sgd)

In [ ]:
all_metrics.append(['Stochastic_Gradient_Descent', mean_squared_error(y_test, y_pred_sgd)])
mean_squared_error(y_test, y_pred_sgd)

Compare with a similar method from the sklearn library:

In [ ]:
sgd_reg = SGDRegressor(max_iter=100000, penalty=None, eta0=0.1, random_state=42)
sgd_reg.fit(X_train_with_c, y_train.ravel())

In [ ]:
print('Parameters returned by SGDRegressor from the sklearn library:')
theta_best_sgd_sklearn = np.array(sgd_reg.coef_)
pd.DataFrame(np.round(theta_best_sgd_sklearn, 3)).T

In [ ]:
X_test_with_c = np.c_[np.ones((X_test.shape[0], 1)), X_test] 
y_pred_sk_sgd = X_test_with_c.dot(theta_best_sgd_sklearn)

In [ ]:
all_metrics.append(['Sklearn_Stochastic_Gradient_Descent', mean_squared_error(y_test, y_pred_sk_sgd)])
mean_squared_error(y_test, y_pred_sk_sgd)

`Comment:` The sklearn implementation works much more efficiently in terms of computation speed and accuracy (RMSE parameter).

### Mini batch gradient descent House Prices: Advanced Regression Techniques

This modification of the algorithm combines the features of a batch and stochastic implementation.
The mini batch gradient descent calculates gradients on small randomly sampled data samples (__mini-batch__).
The larger the package, the less the wandering of the algorithm with respect to the shortest path from the starting point to the local minimum.
In general, the mini-batch gradient descent is selected closer to the minimum than the stochastic version.
On the other hand, this algorithm is more difficult to get away from local minima, especially when there are a lot of them.
Given a well-chosen training schedule, the mini-batch gradient descent method allows you to reach a minimum faster than the batch version.

`Advantages`: the algorithm is computationally cheaper than the batch version, quickly gives good results, in the case of an irregular loss function (with many local extrema), the algorithm has good chances to jump out of a local minimum and get to a deeper local minimum or even to a global one.

`Disadvantages`: the final answer will not be optimal (the algorithm will not stop even if it gets to a minimum), it will most likely just be" good ".

We write our own implementation of the algorithm.

In [ ]:
def mb_gradient_descent(x, y, mu, params, num_epochs, num_iter, batch_size=0.2, lf='MSE', prnt=False):
    """
    The function implements a mini-batch gradient descent algorithm.
    mu - learning rate
    params - number of parameters, including free parameter
    num_iter - the number of iterations (int) in the era
    batch_size - packet size for one iteration
    prnt - print or not computation
    lf - loss function, default 'MSE', you can select 'MAE'
    """
    
    n = x.shape[0] #количество наблюдений в выборке
    theta = np.ones(params).reshape(params,1) # [ 1.  1.  1.] - начальные значения коэффициентов пусть будут равны 1
    y = y.values.reshape(-1,1)
    
    for epoch in range(num_epochs):
        for iteration in range(num_iter):
            random_index = list(set(np.random.randint(200, size=round(n*batch_size)).tolist()))
            x_rand = x[random_index, :].reshape(len(random_index), params)
            y_rand = y[random_index, :]

            hypothesis = np.dot(x_rand, theta) # матричное произведение
            loss = hypothesis - y_rand # значение остатка

            if lf=='MSE':
                J = np.sum(loss ** 2) / n  # функция потерь (квадраты)
                if prnt and (iteration % 10000)==0:
                    print( "epoch %s | iter %s | MSE: %.3f" % (epoch, iteration, J) )

            elif lf=='MAE':
                J = np.sum(abs(loss)) / n  # функция потерь (модули)
                if prnt and (iteration % 10000)==0:
                    print( "iter %s | MAE: %.3f" % (iteration, J) )

            gradient = np.dot(x_rand.transpose(), loss) * 2 / n  
            theta = theta - mu * gradient  # update
    
    return (theta)

Let's run our algorithm for 50 epochs with 2000 iterations (only 100,000 times) and measure the execution time.

In [ ]:
%%time
theta_best_mbgs = mb_gradient_descent(X_train_with_c, y_train, mu=0.001, params=X_train_with_c.shape[1], num_epochs=50, num_iter=2000, batch_size=0.2, lf='MSE', prnt=True)
pd.DataFrame(np.round(theta_best_mbgs, 3)).T

In [ ]:
X_test_with_c = np.c_[np.ones((X_test.shape[0], 1)), X_test] 
y_pred_mbgs = X_test_with_c.dot(theta_best_mbgs)

In [ ]:
all_metrics.append(['Mini_batch_gradient_descent', mean_squared_error(y_test, y_pred_mbgs)])
mean_squared_error(y_test, y_pred_mbgs)

# Linear regression models with regularization

__Regularization__ is the imposition of certain restrictions on the model in order to avoid retraining.

The simplest example of regularization is to use the 2nd degree of the polynomial when generating polynomial features, but the 2nd.
In addition, you can artificially limit the size of the coefficients of the $\theta$ model.

Regularized regression models include:
    
* Ridge regression ($L_2$ -regulation)
* Lasso regression ($L_1$ regularization)
* Elastic network

### Ridge-regression ($L_2$ -regularization) House Prices: Advanced Regression Techniques

The regularization of the model is realized by adding to the loss function *a regularization term*:

$$\alpha \sum_{i=1}^m \theta_i^2$$

Where:

$\alpha$ - normalization parameter, determines how much we regularize the model. With $\alpha=0$, ridge regression becomes just a linear regression. For $\alpha$ close to 1, the model coefficients (weights) tend to zero;

$\theta_i$ - model parameters from $\theta_1$ to $\theta_m$.


`Note to expression`: *the regularization term* is used in conjunction with the loss function __only__ when training the model. When checking the model you need to use an irregular measure of errors. This is also true for the other regularized regression models described below.

The parameter with the free term $\theta_0$ is not regularized

The ridge regression loss function may have the form:

$$ J(\theta) = MSE(X,h_\theta) + \frac{1}{2}\alpha \sum_{i=1}^m \theta_i^2$$

$$ J(\theta) = RMSE(X,h_\theta) + \frac{1}{2}\alpha \sum_{i=1}^m \theta_i^2$$

$\alpha$ - normalization parameter, determines how much we regularize the model. With $\alpha=0$, ridge regression becomes just a linear regression. With $\alpha$ close to 1, the best possible coefficients (weights) of the model tend to zero.

$\theta_i$ - model parameters from $\theta_1$ to $\theta_m$.

`Note to the expression`: $\theta_0$ (parameter with a free term) is not regularized! The sum $\sum_{i = 1}^m$ starts at 1, not 0.

Of the various regularization options, we will use first the formula.

As a result, the vector of coefficients (weights) of the model is formed into a vector of the form:

$$
\frac{1}{2} \alpha 
\left(\begin{array}{cc} 
(\theta_1^2 + \theta_2^2 + ... + \theta_m^2)^{1/2} \\
\theta_1^2 \\
\theta_2^2 \\
... \\
\theta_m
\end{array}\right) 
$$ 

Where:

$(\theta_1^2 + \theta_2^2 + ... + \theta_m^2)^{1/2}$ - L2-norm of the coefficient vector from $\theta_1$ до $\theta_m$.

Before applying ridge regression, it is recommended to scale the data (bring them to the same dimension), the model is sensitive to the scale of features.

### Ridge regularization example House Prices: Advanced Regression Techniques

In [ ]:
# we create data for training the model in the amount of m
m=100
X = 3 * np.random.rand(m, 1)
y = 1 + 0.5 * X + np.random.randn(m, 1) / 1.5
# create data to test the model in the amount of m
X_new = np.linspace(0, 3, m).reshape(m, 1)
y_new =  1 + 0.5 * X_new + np.random.randn(m, 1) / 1.5

# prepare 6 graphs for visualizing the forecast with a change in alpha
fig = plt.figure(figsize = (14,6))
fig.subplots_adjust(hspace=0.4, wspace=0.4)
# список значений alpha
alpha_list = [0, 0.00001, 0.002, 0.5, 0.8, 1]
for n,i in zip([ni for ni in range(1,len(alpha_list)+1)], alpha_list):
    model = Ridge(i) # sklearn ridge regression model with alpha
    poly_features = PolynomialFeatures(degree=18, include_bias=False) # PolynomialFeatures с степенью 18
    # expanding data for training with polynomial features
    Х_poly_features = poly_features.fit_transform(X)
    # we scale advanced data for training
    X_poly_features_scaled = StandardScaler().fit_transform(Х_poly_features)
    # we scale the target variable for training
    y_scaled = StandardScaler().fit_transform(y)
    
    model.fit(X_poly_features_scaled, y_scaled) # обучаем модель Ridge
    
    # doing the procedure with the data to check (X_new)
    X_new_poly_features = poly_features.fit_transform(X_new) # расширяем
    X_new_poly_features_scaled = StandardScaler().fit_transform(X_new_poly_features) # шкалируем

    y_pred = model.predict(X_new_poly_features_scaled) # делаем предсказание
    
    # plot
    ax = fig.add_subplot(2,3,n)
    # all x and y will be drawn in a scaled version
    ax.plot(StandardScaler().fit_transform(X_new), y_pred, "r", label='X_new', linewidth=4)
    ax.plot(StandardScaler().fit_transform(X_new), StandardScaler().fit_transform(y_new), "b.")
    ax.set_title("alpha={}".format(i), fontsize = 10)
    ax.set_xlabel("X")
    ax.set_ylabel("y")
    ax.set_ylim((-3, 3))
    fig.suptitle("Pic. Forecast change with increasing coefficient limit", \
            fontsize = 14, y = 1.03)
fig.tight_layout()
fig.show()

In [ ]:
# add a unit column to the left of the observation matrix (theta zero parameter will be multiplied by 1)
X_train_with_c = np.c_[np.ones((X_train.shape[0], 1)), X_train] 
theta_best_analytic = np.linalg.inv(X_train_with_c.T.dot(X_train_with_c)).dot(X_train_with_c.T).dot(y_train)
pd.DataFrame(theta_best_analytic).T

In [ ]:
# set the alpha parameter
alpha = 0.5
# create matrix A
A = np.zeros((theta_best_analytic.shape[0],theta_best_analytic.shape[0]))
for s,c in zip([si for si in range(A.shape[0])], [ci for ci in range(A.shape[1])]):
    A[s,c] = alpha
A[0,0] = 0
A

In [ ]:
theta_best_ridge = np.linalg.inv(X_train_with_c.T.dot(X_train_with_c) + alpha*A).dot(X_train_with_c.T).dot(y_train)
pd.DataFrame(np.round(theta_best_ridge, 3)).T

In [ ]:
X_test_with_c = np.c_[np.ones((X_test.shape[0], 1)), X_test] 
pd.DataFrame(X_test_with_c[:5,:])

In [ ]:
# make a prediction using our model
y_pred_ridge = X_test_with_c.dot(theta_best_ridge)
all_metrics.append(['Ridge', mean_squared_error(y_test, y_pred_ridge)])
mean_squared_error(y_test, y_pred_ridge)

Now compare with the sklearn implementation:

In [ ]:
model_ridge = Ridge(alpha=1)
model_ridge.fit(X_train, y_train)

In [ ]:
all_metrics.append(['sklearn_Ridge', mean_squared_error(y_test, model_ridge.predict(X_test))])
mean_squared_error(y_test, model_ridge.predict(X_test))

`Note:` native and off-the-shelf implementations work with roughly the same RMSE

# Lasso regression House Prices: Advanced Regression Techniques
#### ($L_1$ -regularization, least absolute shrinkage and selection operator regression regression)  House Prices: Advanced Regression Techniques

Lasso regression is another type of regularized regression models in which the L1 norm of the weight vector $\theta$ is used instead of 1/2 the norm square of the weight vector $\theta$ (as in the case of ridge regression).

Lasso regression loss function:

$$ J(\theta) = MSE(X,h_\theta) + \alpha \sum_{i=1}^m |\theta_i|$$

Where:

$\alpha$ - normalization parameter, determines how much we regularize the model. With $\alpha=0$, the lasso regression becomes just a linear regression. When $\alpha$ is close to 1, the coefficients (weights) of the model tend to zero (the least important signs are reset first).

$\theta_i$ - model parameters from $\theta_1$ to $\theta_m$. Moreover, $\theta_0$ (parameter with a free term) is not regularized.

`Advantages`: an important feature of lasso regression is its ability to nullify the coefficients ($\theta$) for the least important features, i.e. provide a more sparse (with fewer coefficients) model. The value of the coefficient contributes to the loss function rather large (taken modulo); ideally, to minimize such a contribution, it would be nice to reset the coefficients affecting the forecast accuracy very little. In the case of ridge regression, where the contribution of the value of each coefficient to the loss function is the squared coefficient, it is enough to underestimate the coefficient so that it becomes less than 1 (since numbers from 0 to 1 give a smaller number when squared).

`Disadvantages`: the lasso-regression loss function is not differentiable at zero, you will have to apply gradient descent and do the trick using a subgradient.

Error function with the addition of a vector-subgradient (which we use at points with x's equal to 0):

$$
MSE(X,h_\theta) + \alpha
\left(\begin{array}{cc} 
sign (\theta_1) \\
sign (\theta_2) \\
... \\
sign (\theta_n)
\end{array}\right) 
$$ 

Where:

$sign (\theta_i) = \begin{cases} 
\displaystyle -1,  \text{если $\theta_i$ < 0} \\
\displaystyle 0,  \text{если $\theta_i$ = 0} \\
\displaystyle +1, \text{если $\theta_i$ > 0}
\end{cases}$

In [ ]:
def gradient_descent(mu, x, y, params, numIterations, lf='MSE', prnt=False):
    """
    The function implements a batch gradient descent algorithm.
    mu - learning rate
    params - number of parameters, including free parameter
    numIterations - number of iterations (int)
    prnt - whether or not to print the calculation, if prnt = True, then at every hundredth step the value of the loss function is displayed
    lf - loss function, default 'MSE', you can select 'MAE'
    """
    
    n = x.shape[0] # number of observations in the sample
    theta = np.ones(params).reshape(params,1) # [ 1.  1. 1.] - начальные значения коэффициентов пусть будут равны 1
    x_transpose = x.transpose() # transposed matrix x
    #print('y', y.shape)
    for iter in range( 0, numIterations ):
        hypothesis = np.dot(x, theta) # matrix multiplication
        loss = hypothesis - y.values.reshape(len(y),1) # residue value
        
        if lf=='MSE':
            J = np.sum(loss ** 2) / n  # loss function (squares)
            if prnt and (iter % 10000)==0:
                print( "iter %s | MSE: %.3f" % (iter, J) )
        
        elif lf=='MAE':
            J = np.sum(abs(loss)) / n  # loss function (modules)
            if prnt and (iter % 10000)==0:
                print( "iter %s | MAE: %.3f" % (iter, J) )
        
        gradient = np.dot(x_transpose, loss) * 2 / n         
        theta = theta - mu * gradient  # update
    
    return (theta)

In [ ]:
def gradient_descent_with_subgrad(mu, x, y, alpha, params, numIterations, prnt=False):
    """
    The function implements a batch gradient descent algorithm using a subgradient with x values equal to 0.
    mu - learning rate
    alpha - hyperparameter of the Lasso regression loss function
    params - number of parameters, including free parameter
    numIterations - number of iterations (int)
    prnt - whether or not to print the calculation, 
    if prnt = True, then at every hundredth step the value of the loss function is displayed
    """
    
    n = x.shape[0] # number of observations in the sample
    theta = np.ones(params).reshape(params,1) # [ 1.  1. 1.] - initial values of the coefficients let be equal to 1
    x_transpose = x.transpose() # transposed matrix x
    
    for iter in range( 0, numIterations ):
        hypothesis = np.dot(x, theta) # matrix multiplication
        loss = hypothesis - y.values.reshape(-1,1) # residue value
        
        
        J = np.sum(loss ** 2) / n  + alpha*np.sum(np.abs(theta[1:,:]))# loss function (squares)
        if prnt and (iter % 10000)==0:
            print( "iter %s | MSE + alpha*sum(abs(theta)): %.3f" % (iter, J) )
        
        if np.sum(theta == 0)>0: # if at least one of theta parameters is zero
            gradient = np.dot(x_transpose, loss) * 2 / n # the gradient is initially calculated as usual
            # but we add one more term to the gradient to circumvent the non-differentiability of the loss function at zeros
            term = (gradient > 0)*np.ones(gradient.shape[0]).reshape(gradient.shape[0],1) + \
            (gradient < 0)*np.ones(gradient.shape[0]).reshape(gradient.shape[0],1)*(-1)
            # add this term to the gradient
            gradient = gradient + alpha*term.reshape(gradient.shape[0], 1)
        else:
            gradient = np.dot(x_transpose, loss) * 2 / n # in other cases, the gradient is calculated as usual
            
        theta = theta - mu * gradient  # update
    
    return (theta)

In [ ]:
%%time
# add a unit column to the left of the observation matrix (theta zero parameter will be multiplied by 1)
X_train_with_c = np.c_[np.ones((X_train.shape[0], 1)), X_train] 
# run the gradient descent algorithm
theta_best_lasso = gradient_descent_with_subgrad(0.0001, X_train_with_c, y_train, 0.5, params=X_train_with_c.shape[1], numIterations=100000, prnt=True)

In [ ]:
pd.DataFrame(np.round(theta_best_lasso, 3)).T

Now you can use our trained model for forecasts. Let's make a forecast for test.

In [ ]:
X_test_with_c = np.c_[np.ones((X_test.shape[0], 1)), X_test] 
pd.DataFrame(X_test_with_c[:5,:])

In [ ]:
# make a prediction using our model
y_pred_lasso = X_test_with_c.dot(theta_best_lasso)

In [ ]:
all_metrics.append(['Lasso', mean_squared_error(y_test, y_pred_lasso)])
mean_squared_error(y_test, y_pred_lasso)

Compare to sklearn implementation

In [ ]:
lasso_reg = Lasso(alpha=0.5)
lasso_reg.fit(X_train, y_train)
all_metrics.append(['sklearn_Lasso', mean_squared_error(y_test, lasso_reg.predict(X_test))])
mean_squared_error(y_test, lasso_reg.predict(X_test))

# Elastic Net House Prices: Advanced Regression Techniques

An elastic net is a combination of ridge and lasso regression.

Elastic Net Loss Function:

$$ J(\theta) = MSE(X,h_\theta) + r\alpha \sum_{i=1}^m |\theta_i| + \frac{1-r}{2}\alpha \sum_{i=1}^m \theta_i ^2$$

Where:

$r$ is a hyperparameter that controls the proportion between L2 and L1 regularization.

For $r = 0$, an elastic network becomes just a ridge regression, and for $r = 1$ it becomes a lasso regression.

We implement our own algorithm:

In [ ]:
def gradient_descent_with_subgrad_for_elastic_net(mu, x, y, alpha, r, params, numIterations, prnt=False):
    """
    The function implements a batch gradient descent algorithm using a subgradient with x values equal to 0.
    mu - learning rate
    alpha - hyperparameter of the Lasso regression loss function
    params - number of parameters, including free parameter
    numIterations - number of iterations (int)
    prnt - whether or not to print the calculation, if prnt = True, 
    then at every hundredth step the value of the loss function is displayed
    """
    
    n = x.shape[0] # number of observations in the sample
    theta = np.ones(params).reshape(params,1) # [ 1.  1. 1.] - initial values of the coefficients let be equal to 1
    x_transpose = x.transpose() # transposed matrix x
    
    for iter in range( 0, numIterations ):
        hypothesis = np.dot(x, theta) # matrix multiplication
        loss = hypothesis - y.values.reshape(-1,1) # residue value
        
        J = np.sum(loss ** 2) / n  + r*alpha*np.sum(np.abs(theta[1:,:])) + (1-r)/2 * alpha*np.sum(theta[1:,:]**2)
        if prnt and (iter % 10000)==0:
            print( "iter %s | MSE+r*alpha*sum(abs(theta))+(1-r)/2*alpha(theta^2): %.3f" % (iter, J) )
        
        if np.sum(theta == 0)>0: # if at least one of theta parameters is zero
            gradient = np.dot(x_transpose, loss) * 2 / n # the gradient is initially calculated as usual
            # but we add one more term to the gradient to circumvent the non-differentiability of the loss function at zeros
            term = (gradient > 0)*np.ones(gradient.shape[0]).reshape(gradient.shape[0],1) + \
            (gradient < 0)*np.ones(gradient.shape[0]).reshape(gradient.shape[0],1)*(-1)
            # add this term to the gradient
            gradient = gradient + alpha*term.reshape(gradient.shape[0], 1)
        else:
            gradient = np.dot(x_transpose, loss) * 2 / n # in other cases, the gradient is calculated as usual
            
        theta = theta - mu * gradient  # update
    
    return (theta)

In [ ]:
%%time
# add a unit column to the left of the observation matrix (theta zero parameter will be multiplied by 1)
X_train_with_c = np.c_[np.ones((X_train.shape[0], 1)), X_train] 
# run the gradient descent algorithm
theta_best_enet = gradient_descent_with_subgrad_for_elastic_net(0.0001, X_train_with_c, y_train, 0.5, 0.5, params=X_train_with_c.shape[1], numIterations=100000, prnt=True)

In [ ]:
pd.DataFrame(np.round(theta_best_enet, 3)).T

In [ ]:
# take test data
# add left column vector with units
X_test_with_c = np.c_[np.ones((X_test.shape[0], 1)), X_test] 
pd.DataFrame(X_test_with_c[:5,:])

In [ ]:
# make a prediction using our model
y_pred_elnet = X_test_with_c.dot(theta_best_enet)

In [ ]:
all_metrics.append(['Elastic_net', mean_squared_error(y_test, y_pred_elnet)])
mean_squared_error(y_test, y_pred_elnet)

In [ ]:
elastic_net = ElasticNet(alpha=0.5, l1_ratio=0.5, random_state=42)
elastic_net.fit(X_train, y_train)
all_metrics.append(['sklearn_Elastic_net', mean_squared_error(y_test, elastic_net.predict(X_test))])
mean_squared_error(y_test, elastic_net.predict(X_test))

# Conclusions:

##### Implementation of design work tasks:

To put into practice the following regression models:

* Linear regression. 
* Ridge regression. 
* Lasso regression. 
* Elastic network. 

To apply optimization methods:

* Analytical solutions. 
* Gradient descent:
* batch. 
* stochastic. 
* mini batch. 
    

Collect and prepare data, solve the problem of comparability (identical information in different sources is indicated by different names, often measurements are made in different units) and incomplete data (data are not available for long periods) in the following ways:

* Processing of data provided by the customer (work with missing values, aggregation of indicators by periods). 
* Enrichment of data with data obtained by the API from open sources. 
* Designing signs that operate on different phases of the price of houses. 

# Choose the best algorithm for House Prices: Advanced Regression Techniques

In [ ]:
pd.DataFrame(all_metrics, columns=['Algo', 'RMSE']).sort_values('RMSE')

# Predict 

### We will use our own ridge regression algorithm.

In [ ]:
test_df_with_c = np.c_[np.ones((test_df.shape[0], 1)), test_df] 
pd.DataFrame(test_df_with_c[:5,:])

In [ ]:
log_predict = test_df_with_c.dot(theta_best_ridge)

In [ ]:
np.exp(log_predict)[:5]

In [ ]:
sub_df['SalePrice'] = np.round(np.exp(log_predict), 0)
sub_df.head()

In [ ]:
sub_df.to_csv('submission.csv',index=False)

Thank you!